In [4]:
pip install nltk

  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd

In [5]:
train_corpus = pd.read_csv('project2/project2_training_data.txt', delimiter='\n', header=None,names=['sentences'])

In [6]:
train_corpus

,sentences
0,Merrill Lynch analyst Campbell Morgan upgraded...
1,Eriikka S+Âderstr+Âm has previously held sever...
2,The webcast may be followed online on the comp...
3,"Typical end-uses include roof structures , flo..."
4,The sale will be finalized in September or Oct...
...,...
1806,With this appointment Kaupthing Bank aims to f...
1807,Jon Risfelt has previously held operational ex...
1808,The group intends to relocate warehouse and of...
1809,"The contract includes software licences , appl..."


In [7]:
train_labels = pd.read_csv('project2/project2_training_data_labels.txt', delimiter='\n', header=None, names=['labels'])

In [10]:
train_labels

,labels
0,positive
1,neutral
2,neutral
3,neutral
4,neutral
...,...
1806,positive
1807,neutral
1808,neutral
1809,neutral


In [9]:
train_labels.value_counts()

labels  
neutral     1113
positive     456
negative     242
dtype: int64

In [14]:
df = pd.concat([train_corpus, train_labels], axis=1)

In [20]:
pd.set_option('max_colwidth', 600)    
df[df['labels']=='negative']

,sentences,labels
19,"However , the growth margin slowed down due to the financial crisis .",negative
23,"ADPnews - Feb 5 , 2010 - Finnish real estate investor Sponda Oyj HEL : SDA1V said today that it slipped to a net loss of EUR 81.5 million USD 11.8 m in 2009 from a profit of EUR 29.3 million in 2008 .",negative
26,"Alma Media 's operating profit amounted to EUR 11.9 mn , down from EUR 15.0 mn a year earlier .",negative
29,"Operating result showed a loss of EUR 2.9 mn , while a year before , it showed a profit of EUR 0.6 mn .",negative
44,The fair value of the company 's investment properties went down to EUR 2.768 billion at the end of 2009 from EUR 2.916 billion a year earlier .,negative
...,...,...
1756,Consolidated operating profit from continuing operations decreased by 62.3 % to EUR 51.2 mn from EUR 135.7 mn in 2007 .,negative
1764,"Salcomp Oyj , the Finnish maker of mobile phone chargers , Monday posted a EUR1 .49 million loss in the second quarter compared with a 1.70 million profit in the same period the previous year .",negative
1767,Net sales dropped by 6 % year-on-year to EUR 11.9 million .,negative
1773,Finnish business software group AffectoGenimap Oyj said its net profit halved to 1.2 mln euro ( $ 1.5 mln ) in the first nine months of 2006 from 2.2 mln euro ( $ 2.8 mln ) in the same period of 2005 .,negative


### Pre-processing

In [21]:
from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize, TreebankWordTokenizer, TweetTokenizer, MWETokenizer
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [22]:
text='This is an elective course'

text.split()

# text="Let me know your mobile#, if that's fine. The call rate is 1.2p/s 😊️."

# print(word_tokenize(text))


['This', 'is', 'an', 'elective', 'course']

In [23]:
word_tokenize(text)

['This', 'is', 'an', 'elective', 'course']

In [21]:
import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))
print(stops)

{'themselves', 'my', "should've", "needn't", 'and', 'was', 'had', 'at', 'few', 'should', 'am', 'ma', 'too', 're', 'to', 'each', "wasn't", "you'd", 'myself', "isn't", 'for', 'your', 'as', 'until', 'whom', 'their', 've', 'under', 'wasn', 'while', 'shouldn', 'during', 'mightn', 'not', 'most', 'very', 'shan', "mightn't", 'doesn', 'about', 'he', "you've", 'then', 'other', 'nor', 'is', 'where', 'be', 'can', 'doing', 'how', 'being', "doesn't", 'with', 'aren', 'yourself', 'were', 'up', 'have', 'below', "wouldn't", 'a', "don't", 'who', 'between', 'own', 'ain', 'just', 'will', 'didn', 'an', 'hasn', 'won', 'or', 'his', 'by', 'any', 'before', 'hadn', 'having', 'you', 'll', "hadn't", 'only', 'it', 'these', 'o', 'did', 'theirs', 'into', 'yourselves', 'her', 'when', 'all', 'once', "aren't", 'himself', 'on', 'in', 'than', 'because', 'she', 'yours', 'so', 'they', 'no', 'don', 'through', 'we', "it's", 'both', 'here', 't', "didn't", "mustn't", 'after', 'if', 'now', "hasn't", 'those', "shan't", 'this', 'i

In [24]:
stops = sorted(list(stops))

In [25]:
stops

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

"aren't",  'above', 'couldn', "couldn't, 'didn', "didn't",'doesn',"doesn't", 'don',"don't", 'below', 'before', 'down','hadn',"hadn't",'hasn',"hasn't", 'haven',"haven't","isn't", 'isn', 'mightn',"mightn't", 'mustn', "mustn't", 'needn', "needn't", 'more', 'further','from','no','nor','not', 'over', 'shan',"shan't", 'shouldn',"shouldn't", 'to','under', 'up','wasn',"wasn't",'weren',"weren't", 'won',"won't",'wouldn', "wouldn't",



